# Bright By Text: Outgoing Messages Analysis

## Setup and Data Import

In [2]:
# Widen notebook

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
# Import libraries necessary
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import pandas as pd
from time import time
import datetime
from IPython.display import display
import re
%matplotlib inline
import psycopg2
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:.3f}'.format

In [19]:
# Connect to PostgeSQL

try:
    conn = psycopg2.connect(user = "pbiusr1",
                                  password = "RE_^V%Gj@EL6R!G",
                                  host = "app.brightbytext.org",
                                  port = "5432",
                                  database = "bbtapi")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

In [20]:
sql = '''
    select
    om.message_id,
    om.created_at as outbound_message_created_at,
    om.body as outbound_message_body,
    om.partner_id as outbound_message_partner_id,
    mt.name as message_type,
    s.id as subscriber_id,
    s.created_at,
    s.signedup_at,
    s.deactivated_at,
    s.carrier_name,
    z.code as subscriber_zip_code,
    z.city as subscriber_city,
    t.default_offset as timezone_default_offset,
    t.iana_name as timezone_name,
    s_state.name as subscriber_state,
    ss.name as subscriber_source,
    sst.name as subscriber_status,
    l.name as subscriber_language,
    sdr.name as subscriber_deactivation_method,
    p.name as partner_name,
    p.created_at as partner_created_date,
    p.is_active as partner_is_active,	
    p_state.name as partner_state
    
from public.outbound_messages as om
left join public.message_types as mt
on om.message_type_id = mt.id 
left join public.v_subscribers as s
on om.subscriber_id = s.id
left join public.zip_codes as z 
on z.id = s.zip_id
left join public.timezones t
on z.time_zone_id = t.id
left join public.states as s_state
on s_state.id = z.state_id
left join public.subscriber_sources as ss 
on ss.id = s.source_id
left join public.subscriber_statuses as sst 
on sst.id = s.status_id
left join public.languages as l
on l.id = s.language_id
left join public.subscriber_deactivation_reasons as sdr 
on sdr.id = s.deactivation_reason_id
left join public.partners as p
on p.id = s.partner_id
left join public.partner_types as pt 
on pt.id = p.type_id
left join public.states as p_state
on p_state.id = p.state_id
'''
messages = pd.read_sql_query(sql, conn)
conn = None

In [29]:
messages

,message_id,outbound_message_created_at,outbound_message_body,outbound_message_partner_id,message_type,subscriber_id,created_at,signedup_at,deactivated_at,carrier_name,subscriber_zip_code,subscriber_city,timezone_default_offset,timezone_name,subscriber_state,subscriber_source,subscriber_status,subscriber_language,subscriber_deactivation_method,partner_name,partner_created_date,partner_is_active,partner_state
0,2b4a9171-9153-47d6-b149-749dc78f099d,2020-01-13 17:05:27.836324,Help children get ready to read: make eye cont...,b6a20a01-a55e-481c-9878-7c0ce616374a,normal,79bd2a14-1fc2-4c33-86ad-4b148f9d188b,2019-09-08 20:58:57.048526,2019-09-08 21:02:30.683962,NaN,US Cellular Corp.,53807.0,Cuba City,-6.0,America/Chicago,Wisconsin,Text SignUp,activated,English,NaN,Wisconsin Public Television,2019-04-17 19:52:45.009641,True,Wisconsin
1,38188594-8d98-4966-bfba-25b0122b7af4,2020-01-13 17:05:28.024023,MATH: Use shapes to make patterns--games like ...,62c3f855-e888-41a5-ab0b-1dd69a01dd3d,normal,37ce064c-f43e-41aa-b193-74a9fbd458ec,2019-05-07 20:34:54.218836,2019-05-07 20:34:54.207131,NaN,Cricket Wireless - ATT - SVR,78521.0,Brownsville,-6.0,America/Chicago,Texas,Web SignUp,activated,English,NaN,United Way of Southern Cameron County - Browns...,2018-08-28 20:59:07.931146,True,Texas
2,152bc534-4cf6-4675-98cd-b7e3941fc69f,2020-01-13 17:05:28.056393,"Favorite Flavors: As you eat with your child, ...",24bf3729-46b3-47fa-ba19-6786f208758b,normal,51438d4e-a61f-40de-a924-d21155b3ba9d,2018-02-20 01:20:40.241215,2018-02-20 01:20:40.241215,NaN,"Sprint Spectrum, L.P.",75093.0,Plano,-6.0,America/Chicago,Texas,Text SignUp,activated,English,NaN,KERA - Dallas,2018-08-28 21:01:36.642848,True,Texas
3,11b0a16a-d3f5-4f8f-ab15-71a691d26dac,2020-01-13 17:05:28.060618,Ask your child lots of questions and give him ...,24bf3729-46b3-47fa-ba19-6786f208758b,normal,d603f242-60e9-437e-a8af-44b6ef92fa3b,2018-11-08 23:26:10.598509,2018-11-08 23:26:10.598509,NaN,AT&T Wireless,77450.0,Katy,-6.0,America/Chicago,Texas,Text SignUp,activated,English,NaN,KERA - Dallas,2018-08-28 21:01:36.642848,True,Texas
4,5743832e-f78a-4849-be4c-a755844e9f5e,2020-01-13 17:05:28.112474,Children’s feeding behaviors will change as th...,24bf3729-46b3-47fa-ba19-6786f208758b,normal,f004e216-d587-43a2-a863-34162de26d4d,2018-02-23 21:04:19.347275,2018-02-23 21:04:19.347275,2020-02-13 19:33:13.978850,AT&T Wireless,76001.0,Arlington,-6.0,America/Chicago,Texas,Text SignUp,deactivated,English,STOP,KERA - Dallas,2018-08-28 21:01:36.642848,True,Texas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3554657,de14d62e-7be9-464a-bfc8-8e66e0398f81,2020-01-16 16:04:05.153884,Often a child's environment has an impact on h...,87d65e72-a157-4cd9-b7a4-f1e6ff052f42,normal,87f7d955-e9ce-482c-a98c-dfe9402d6de3,2018-01-30 15:18:53.557824,2018-01-30 15:18:53.557824,NaN,Verizon Wireless,46131.0,Franklin,-5.0,America/Indiana/Indianapolis,Indiana,Text SignUp,activated,English,NaN,"WFYI - Indianapolis, IN",2018-08-28 21:00:49.021838,True,Indiana
3554658,b74a10ba-0bb6-47b8-8923-8aa2342a8780,2020-01-16 16:04:05.163776,Texture Tryout: Experiment with your child usi...,97f6bf86-bbab-4e74-b3a9-3940bbc340a9,normal,66a85890-445d-4bbd-9106-06541eb466ce,2017-12-01 16:30:15.521634,2017-12-01 16:30:15.521634,NaN,Verizon Wireless,28451.0,Leland,-5.0,America/New_York,North Carolina,Text SignUp,activated,English,NaN,UNC-TV- North Carolina,2018-08-28 21:01:13.109946,True,North Carolina
3554659,9ac21496-026d-401b-9ee2-1910472da722,2020-01-16 16:04:05.178545,Your child is ready to learn. All you need is ...,d4d696af-37ff-427e-8035-2cb1bd630495,normal,82985fd2-c567-47f3-ba69-a25f291b3640,2017-06-14 00:01:59.066572,2017-06-14 00:01:59.066572,NaN,Verizon Wireless,11701.0,Amityville,-5.0,America/New_York,New York,Text SignUp,activated,English,NaN,Bright by Three,2018-11-30 19:20:34.357527,True,Colorado
3554660,f977d09f-3b28-4430-94bf-b5a9a5ae3701,2020-01-16 16:04:05.245447,Colors We Like: Tell yo

In [ ]:
# Download the messages dataset

messages.to_csv(r'/Users/lisafan/Desktop/BBT/bbt_messages_data.csv', index=False)  

In [4]:
# Load the messages dataset
messages = pd.read_csv(r'/Users/lisafan/Desktop/BBT/bbt_messages_data.csv')

Note that this table only includes outbound messages, and does not include inbound messages.

## Initial Data Exploration and Cleaning

In [5]:
messages['outbound_message_created_at'] = pd.to_datetime(messages['outbound_message_created_at']) # Change message timestamp to datetime format
messages['subscriber_zip_code'] = messages['subscriber_zip_code'].fillna(-1) # Fill NAs in order to convert to integers
messages['subscriber_zip_code'] = messages['subscriber_zip_code'].astype(int).astype(str).str.zfill(5) # Pad zip codes with 0s
messages['subscriber_zip_code'] = messages['subscriber_zip_code'].replace('000-1', np.nan) # Fill NAs again

In [6]:
messages.info(null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3554662 entries, 0 to 3554661
Data columns (total 23 columns):
message_id                        3554662 non-null object
outbound_message_created_at       3554662 non-null datetime64[ns]
outbound_message_body             3554662 non-null object
outbound_message_partner_id       3554662 non-null object
message_type                      3554662 non-null object
subscriber_id                     3554662 non-null object
created_at                        3554662 non-null object
signedup_at                       3538637 non-null object
deactivated_at                    84419 non-null object
carrier_name                      3553374 non-null object
subscriber_zip_code               3376811 non-null object
subscriber_city                   3376811 non-null object
timezone_default_offset           3376811 non-null float64
timezone_name                     3376811 non-null object
subscriber_state                  3376811 non-null object
subscriber

In [7]:
messages['message_type'].value_counts(normalize = True)

normal      0.753
scheduled   0.134
national    0.058
survey      0.033
signup      0.020
Name: message_type, dtype: float64

Of BBT's outgoing messages:
- 75% are Normal (content messages sent by the system based on DOB) - I think these are also referred to as Program messages
- 13% are Scheduled (messages sent by partners with info about local events, based on age and zip code)
- 6% are National (scheduled messages sent to subscribers by BBT, including those who did not provide their zip code)
- 3% are Signup (messages sent to subscribers during signup)
- 2% are Survey (links to surveys sent to subscribers)

Another thing to note is that BBT's `outbound_messages` table seems to change a lot over time. Today (March 15) there are approximately 3.5 million rows in this table, while several days ago, there were 9 million rows in the table.

## Timing of Messages

### Adjust message timestamp data to reflect local time zones

In [8]:
min_date = messages['outbound_message_created_at'].dt.date.min()
max_date = messages['outbound_message_created_at'].dt.date.max()

print('The outbound messages dataset includes text messages from {} to {}.'.format(min_date, max_date))

The outbound messages dataset includes text messages from 2019-11-10 to 2020-03-15.


In [9]:
messages['timezone_default_offset'].value_counts()

-7.000     1271703
-6.000      946150
-5.000      924543
-8.000      223685
-9.000       10022
-10.000        670
-4.000          38
Name: timezone_default_offset, dtype: int64

In [10]:
# Cities in Arizona
pd.DataFrame(messages[(messages['subscriber_state']) == 'Arizona']['subscriber_city'].value_counts().sort_index())

,subscriber_city
Ajo,42
Amado,10
Apache Junction,1557
Arizona City,182
Ash Fork,30
Avondale,7949
Bagdad,196
Bapchule,69
Black Canyon City,116
Blue Gap,58


In [11]:
navajo = ['Chinle', 'Fort Defiance', 'Kayenta', 'Leupp', 'Many Farms', 'Pinon', 'Tonalea', 'Tuba City'] # Cities in Navajo Nation

In [12]:
def dst(row):
    if (row['timezone_default_offset'] == -5) & (row['outbound_message_created_at'] >=  pd.Timestamp('2020-03-08T07')):
        return row['timezone_default_offset'] + 1 # Eastern Time Zone Daylight Savings Adjustment
    elif (row['timezone_default_offset'] == -6) & (row['outbound_message_created_at'] >=  pd.Timestamp('2020-03-08T08')):
        return row['timezone_default_offset'] + 1 # Central Time Zone Daylight Savings Adjustment
    elif (row['timezone_default_offset'] == -7) & (row['outbound_message_created_at'] >=  pd.Timestamp('2020-03-08T09')) & (row['subscriber_state'] != 'Arizona'):
        return row['timezone_default_offset'] + 1 # Mountain Time Zone Daylight Savings Adjustment
    elif (row['timezone_default_offset'] == -7) & (row['outbound_message_created_at'] >=  pd.Timestamp('2020-03-08T09')) & (row['subscriber_state'] == 'Arizona') & (row['subscriber_city'] in navajo):
        return row['timezone_default_offset'] + 1 # Mountain Time Zone Daylight Savings Adjustment for Navajo Nation
    elif (row['timezone_default_offset'] == -8) & (row['outbound_message_created_at'] >=  pd.Timestamp('2020-03-08T10')) & (row['subscriber_state'] != 'Arizona'):
        return row['timezone_default_offset'] + 1 # Pacific Time Zone Daylight Savings Adjustment
    elif (row['timezone_default_offset'] == -9) & (row['outbound_message_created_at'] >=  pd.Timestamp('2020-03-08T11')):
        return row['timezone_default_offset'] + 1 # Alaska Time Zone Daylight Savings Adjustment
    else:
        return row['timezone_default_offset']

In [13]:
messages['timezone_default_offset_adj'] = messages.apply(dst, axis=1)

Puerto Rico (`timezone_default_offset` = -4) and Hawaii (`timezone_default_offset` = -10) do not change their clocks for Daylight Savings. Arizona needs to be accounted for separately because they don't change their clocks for Daylight Savings, with the exception of the Navajo Nation.

In [14]:
# Adjust message time stamps for timezones

messages['timezone_default_offset_adj'] = pd.to_timedelta(messages['timezone_default_offset_adj'],'h')
messages['outbound_message_created_at_adj'] = messages['outbound_message_created_at'] + messages['timezone_default_offset_adj']
messages

,message_id,outbound_message_created_at,outbound_message_body,outbound_message_partner_id,message_type,subscriber_id,created_at,signedup_at,deactivated_at,carrier_name,subscriber_zip_code,subscriber_city,timezone_default_offset,timezone_name,subscriber_state,subscriber_source,subscriber_status,subscriber_language,subscriber_deactivation_method,partner_name,partner_created_date,partner_is_active,partner_state,timezone_default_offset_adj,outbound_message_created_at_adj
0,2b4a9171-9153-47d6-b149-749dc78f099d,2020-01-13 17:05:27.836324,Help children get ready to read: make eye cont...,b6a20a01-a55e-481c-9878-7c0ce616374a,normal,79bd2a14-1fc2-4c33-86ad-4b148f9d188b,2019-09-08 20:58:57.048526,2019-09-08 21:02:30.683962,NaN,US Cellular Corp.,53807,Cuba City,-6.000,America/Chicago,Wisconsin,Text SignUp,activated,English,NaN,Wisconsin Public Television,2019-04-17 19:52:45.009641,True,Wisconsin,-1 days +18:00:00,2020-01-13 11:05:27.836324
1,38188594-8d98-4966-bfba-25b0122b7af4,2020-01-13 17:05:28.024023,MATH: Use shapes to make patterns--games like ...,62c3f855-e888-41a5-ab0b-1dd69a01dd3d,normal,37ce064c-f43e-41aa-b193-74a9fbd458ec,2019-05-07 20:34:54.218836,2019-05-07 20:34:54.207131,NaN,Cricket Wireless - ATT - SVR,78521,Brownsville,-6.000,America/Chicago,Texas,Web SignUp,activated,English,NaN,United Way of Southern Cameron County - Browns...,2018-08-28 20:59:07.931146,True,Texas,-1 days +18:00:00,2020-01-13 11:05:28.024023
2,152bc534-4cf6-4675-98cd-b7e3941fc69f,2020-01-13 17:05:28.056393,"Favorite Flavors: As you eat with your child, ...",24bf3729-46b3-47fa-ba19-6786f208758b,normal,51438d4e-a61f-40de-a924-d21155b3ba9d,2018-02-20 01:20:40.241215,2018-02-20 01:20:40.241215,NaN,"Sprint Spectrum, L.P.",75093,Plano,-6.000,America/Chicago,Texas,Text SignUp,activated,English,NaN,KERA - Dallas,2018-08-28 21:01:36.642848,True,Texas,-1 days +18:00:00,2020-01-13 11:05:28.056393
3,11b0a16a-d3f5-4f8f-ab15-71a691d26dac,2020-01-13 17:05:28.060618,Ask your child lots of questions and give him ...,24bf3729-46b3-47fa-ba19-6786f208758b,normal,d603f242-60e9-437e-a8af-44b6ef92fa3b,2018-11-08 23:26:10.598509,2018-11-08 23:26:10.598509,NaN,AT&T Wireless,77450,Katy,-6.000,America/Chicago,Texas,Text SignUp,activated,English,NaN,KERA - Dallas,2018-08-28 21:01:36.642848,True,Texas,-1 days +18:00:00,2020-01-13 11:05:28.060618
4,5743832e-f78a-4849-be4c-a755844e9f5e,2020-01-13 17:05:28.112474,Children’s feeding behaviors will change as th...,24bf3729-46b3-47fa-ba19-6786f208758b,normal,f004e216-d587-43a2-a863-34162de26d4d,2018-02-23 21:04:19.347275,2018-02-23 21:04:19.347275,2020-02-13 19:33:13.978850,AT&T Wireless,76001,Arlington,-6.000,America/Chicago,Texas,Text SignUp,deactivated,English,STOP,KERA - Dallas,2018-08-28 21:01:36.642848,True,Texas,-1 days +18:00:00,2020-01-13 11:05:28.112474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3554657,de14d62e-7be9-464a-bfc8-8e66e0398f81,2020-01-16 16:04:05.153884,Often a child's environment has an impact on h...,87d65e72-a157-4cd9-b7a4-f1e6ff052f42,normal,87f7d955-e9ce-482c-a98c-dfe9402d6de3,2018-01-30 15:18:53.557824,2018-01-30 15:18:53.557824,NaN,Verizon Wireless,46131,Franklin,-5.000,America/Indiana/Indianapolis,Indiana,Text SignUp,activated,English,NaN,"WFYI - Indianapolis, IN",2018-08-28 21:00:49.021838,True,Indiana,-1 days +19:00:00,2020-01-16 11:04:05.153884
3554658,b74a10ba-0bb6-47b8-8923-8aa2342a8780,2020-01-16 16:04:05.163776,Texture Tryout: Experiment with your child usi...,97f6bf86-bbab-4e74-b3a9-3940bbc340a9,normal,66a85890-445d-4bbd-9106-06541eb466ce,2017-12-01 16:30:15.521634,2017-12-01 16:30:15.521634,NaN,Verizon Wireless,28451,Leland,-5.000,America/New_York,North Carolina,Text SignUp,activated,English,NaN,UNC-TV- North Carolina,2018-08-28 21:01:13.109946,True,North Carolina,-1 days +19:00:00,2020-01-16 11:04:05.163776
3554659,9ac21496-026d-401b-9ee2-1910472da722,2020-01-16 16:04:05.178545,Your child is ready to learn. All you need is ...,d4d696af-

In [15]:
messages['outbound_message_created_at_adj'] = pd.to_datetime(messages['outbound_message_created_at_adj']) 
day_dict = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
messages['outbound_message_created_at_day'] = messages['outbound_message_created_at_adj'].dt.dayofweek.map(day_dict)
messages

,message_id,outbound_message_created_at,outbound_message_body,outbound_message_partner_id,message_type,subscriber_id,created_at,signedup_at,deactivated_at,carrier_name,subscriber_zip_code,subscriber_city,timezone_default_offset,timezone_name,subscriber_state,subscriber_source,subscriber_status,subscriber_language,subscriber_deactivation_method,partner_name,partner_created_date,partner_is_active,partner_state,timezone_default_offset_adj,outbound_message_created_at_adj,outbound_message_created_at_day
0,2b4a9171-9153-47d6-b149-749dc78f099d,2020-01-13 17:05:27.836324,Help children get ready to read: make eye cont...,b6a20a01-a55e-481c-9878-7c0ce616374a,normal,79bd2a14-1fc2-4c33-86ad-4b148f9d188b,2019-09-08 20:58:57.048526,2019-09-08 21:02:30.683962,NaN,US Cellular Corp.,53807,Cuba City,-6.000,America/Chicago,Wisconsin,Text SignUp,activated,English,NaN,Wisconsin Public Television,2019-04-17 19:52:45.009641,True,Wisconsin,-1 days +18:00:00,2020-01-13 11:05:27.836324,Monday
1,38188594-8d98-4966-bfba-25b0122b7af4,2020-01-13 17:05:28.024023,MATH: Use shapes to make patterns--games like ...,62c3f855-e888-41a5-ab0b-1dd69a01dd3d,normal,37ce064c-f43e-41aa-b193-74a9fbd458ec,2019-05-07 20:34:54.218836,2019-05-07 20:34:54.207131,NaN,Cricket Wireless - ATT - SVR,78521,Brownsville,-6.000,America/Chicago,Texas,Web SignUp,activated,English,NaN,United Way of Southern Cameron County - Browns...,2018-08-28 20:59:07.931146,True,Texas,-1 days +18:00:00,2020-01-13 11:05:28.024023,Monday
2,152bc534-4cf6-4675-98cd-b7e3941fc69f,2020-01-13 17:05:28.056393,"Favorite Flavors: As you eat with your child, ...",24bf3729-46b3-47fa-ba19-6786f208758b,normal,51438d4e-a61f-40de-a924-d21155b3ba9d,2018-02-20 01:20:40.241215,2018-02-20 01:20:40.241215,NaN,"Sprint Spectrum, L.P.",75093,Plano,-6.000,America/Chicago,Texas,Text SignUp,activated,English,NaN,KERA - Dallas,2018-08-28 21:01:36.642848,True,Texas,-1 days +18:00:00,2020-01-13 11:05:28.056393,Monday
3,11b0a16a-d3f5-4f8f-ab15-71a691d26dac,2020-01-13 17:05:28.060618,Ask your child lots of questions and give him ...,24bf3729-46b3-47fa-ba19-6786f208758b,normal,d603f242-60e9-437e-a8af-44b6ef92fa3b,2018-11-08 23:26:10.598509,2018-11-08 23:26:10.598509,NaN,AT&T Wireless,77450,Katy,-6.000,America/Chicago,Texas,Text SignUp,activated,English,NaN,KERA - Dallas,2018-08-28 21:01:36.642848,True,Texas,-1 days +18:00:00,2020-01-13 11:05:28.060618,Monday
4,5743832e-f78a-4849-be4c-a755844e9f5e,2020-01-13 17:05:28.112474,Children’s feeding behaviors will change as th...,24bf3729-46b3-47fa-ba19-6786f208758b,normal,f004e216-d587-43a2-a863-34162de26d4d,2018-02-23 21:04:19.347275,2018-02-23 21:04:19.347275,2020-02-13 19:33:13.978850,AT&T Wireless,76001,Arlington,-6.000,America/Chicago,Texas,Text SignUp,deactivated,English,STOP,KERA - Dallas,2018-08-28 21:01:36.642848,True,Texas,-1 days +18:00:00,2020-01-13 11:05:28.112474,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3554657,de14d62e-7be9-464a-bfc8-8e66e0398f81,2020-01-16 16:04:05.153884,Often a child's environment has an impact on h...,87d65e72-a157-4cd9-b7a4-f1e6ff052f42,normal,87f7d955-e9ce-482c-a98c-dfe9402d6de3,2018-01-30 15:18:53.557824,2018-01-30 15:18:53.557824,NaN,Verizon Wireless,46131,Franklin,-5.000,America/Indiana/Indianapolis,Indiana,Text SignUp,activated,English,NaN,"WFYI - Indianapolis, IN",2018-08-28 21:00:49.021838,True,Indiana,-1 days +19:00:00,2020-01-16 11:04:05.153884,Thursday
3554658,b74a10ba-0bb6-47b8-8923-8aa2342a8780,2020-01-16 16:04:05.163776,Texture Tryout: Experiment with your child usi...,97f6bf86-bbab-4e74-b3a9-3940bbc340a9,normal,66a85890-445d-4bbd-9106-06541eb466ce,2017-12-01 16:30:15.521634,2017-12-01 16:30:15.521634,NaN,Verizon Wireless,28451,Leland,-5.000,America/New_York,North Carolina,Text SignUp,activated,English,NaN,UNC-TV- North Carolina,2018-08-28 21:01:13.109946,True,North Carolina,-1 days +19:00:00,2020-01-16 11:04:05.163776,Thursday
3554659,9ac21496-026d-401b-9ee2-1910472da7

### Day of Week Sent

In [16]:
messages['outbound_message_created_at_day'].value_counts(normalize = True)

Thursday    0.184
Tuesday     0.160
Wednesday   0.141
Friday      0.138
Monday      0.130
Saturday    0.124
Sunday      0.123
Name: outbound_message_created_at_day, dtype: float64

Overall, we see the most messages sent on Thursday, followed by Tuesday, with the least amount of messages sent on Sunday. In general, messages are sent more frequently during the weekdays.

In [17]:
messages.groupby('message_type')['outbound_message_created_at_day'].value_counts(normalize = True)

message_type  outbound_message_created_at_day
national      Thursday                          0.651
              Tuesday                           0.349
normal        Sunday                            0.144
              Friday                            0.143
              Wednesday                         0.143
              Thursday                          0.143
              Monday                            0.143
              Tuesday                           0.142
              Saturday                          0.141
scheduled     Wednesday                         0.213
              Friday                            0.192
              Thursday                          0.150
              Monday                            0.142
              Saturday                          0.114
              Tuesday                           0.097
              Sunday                            0.092
signup        Thursday                          0.172
              Wednesday             

- National messages: Most messages are sent out on Thursday, followed by Tuesday. There are no messages sent out on any other day.
- Normal messages: Messages are sent pretty evenly across the seven days of the week. 
- Scheduled messages: The most messages are sent out on Wednesday and Friday. The least number of messages are sent out on Tuesday and Sunday. 
- Signup messages: The most messages are sent out on Thursday. The least number of messages are sent out on Saturday and Sunday. 
- Survey messages: The majority of messages are sent on Tuesday and Thursday, with barely any messages sent on any of the other days. 

### Time of Day Sent

In [18]:
messages['outbound_message_created_at_hour'] = messages['outbound_message_created_at_adj'].dt.hour
messages['outbound_message_created_at_hour'].value_counts(normalize = True)

11.000   0.776
15.000   0.061
14.000   0.027
12.000   0.025
13.000   0.020
10.000   0.019
9.000    0.014
17.000   0.014
16.000   0.012
8.000    0.011
7.000    0.011
19.000   0.004
18.000   0.003
20.000   0.001
6.000    0.000
21.000   0.000
22.000   0.000
23.000   0.000
0.000    0.000
5.000    0.000
1.000    0.000
2.000    0.000
3.000    0.000
4.000    0.000
Name: outbound_message_created_at_hour, dtype: float64

The majority of messages are sent at 11am local time.

In [19]:
pd.set_option('display.max_rows', 100)
pd.options.display.float_format = '{:.3f}'.format
pd.DataFrame(messages.groupby('message_type')['outbound_message_created_at_hour'].value_counts(normalize = True))

outbound_message_created_at_hour
message_type outbound_message_created_at_hour                                  
national     15.000                                                       0.197
             13.000                                                       0.166
             14.000                                                       0.139
             12.000                                                       0.137
             16.000                                                       0.136
             17.000                                                       0.101
             11.000                                                       0.090
             10.000                                                       0.019
             18.000                                                       0.014
             9.000                                                        0.001
             8.000                                                        0.000
normal       11.000                                                       1.000
             13.000                                                       0.000
             12.000                                                       0.000
             10.000                                                       0.000
             9.000                                                        0.000
             8.000                                                        0.000
scheduled    11.000                                                       0.140
             14.000                                                       0.126
             10.000                                                       0.120
             12.000                                                       0.111
             15.000                                                       0.097
             9.000                                                        0.092
             7.000                                                        0.070
             8.000                                                        0.068
             13.000                                                       0.063
             17.000                                                       0.049
             19.000                                                       0.027
             16.000                                                       0.022
             18.000                                                       0.013
             6.000                                                        0.002
             20.000                                                       0.000
             4.000                                                        0.000
             21.000                                                       0.000
signup       11.000                                                       0.121
             12.000                                                       0.094
             13.000                                                       0.094
             10.000                                                       0.082
             14.000                                                       0.072
             9.000                                                        0.072
             16.000                                                       0.065
             8.000                                                        0.063
             15.000                                                       0.060
             17.000                                                       0.047
             7.000                                                        0.044
             18.000                                                       0.035
             20.000                                                       0.034
             19.000                                                       0.031
             21.000                            

- National messages: 20% of national messages are sent at 3pm local time. 97% of all national messages are sent between 11am-5pm local time.
- Normal messages: Normal messages are all sent at 11am local time.
- Scheduled messages: There seems to be more of a random mix of hours during which scheduled messages are sent.
- Signup messages: Signup messages are most often sent at 11am, followed by 12pm and 1pm. 92% of messages are sent between 10am-5pm.
- Survey messages: All survey messages are sent at 3pm local time.

## Frequency of Messages

In [43]:
messages_per_subscriber = messages.groupby('subscriber_zip_code').agg({'subscriber_id':'nunique', 'message_id':'count'}).reset_index()
messages_per_subscriber.columns = ['subscriber_zip_code', 'subscriber_count', 'message_count']
messages_per_subscriber['messages_per_subscriber'] = messages_per_subscriber['message_count'] / messages_per_subscriber['subscriber_count']
messages_per_subscriber

,subscriber_zip_code,subscriber_count,message_count,messages_per_subscriber
0,00501,1,22,22.000
1,00544,1,7,7.000
2,00960,1,38,38.000
3,01011,1,38,38.000
4,01060,1,61,61.000
...,...,...,...,...
7697,99783,1,12,12.000
7698,99801,27,1274,47.185
7699,99803,2,106,53.000
7700,99833,1,17,17.000


In [49]:
print("There is an average of {:,.0f} messages per subscriber sent out per zip code.".format(messages_per_subscriber['messages_per_subscriber'].mean()))

There is an average of 42 messages per subscriber sent out per zip code.


In [52]:
messages_per_subscriber.nlargest(10, 'messages_per_subscriber')

,subscriber_zip_code,subscriber_count,message_count,messages_per_subscriber
496,08221,1,405,405.000
3567,45005,1,251,251.000
5171,73160,1,152,152.000
1301,15955,1,150,150.000
4319,53930,1,149,149.000
3454,42069,1,145,145.000
4545,55923,1,142,142.000
6450,81252,1,142,142.000
2791,30281,1,141,141.000
4466,54805,1,141,141.000


There are users getting as many as 405 messages. Is this overkill? Let's take a closer look.

In [54]:
messages[messages['subscriber_zip_code'] == '08221']['message_type'].value_counts()

normal       344
signup        54
national       3
survey         2
scheduled      2
Name: message_type, dtype: int64

In [55]:
messages[messages['subscriber_zip_code'] == '08221']

,message_id,outbound_message_created_at,outbound_message_body,outbound_message_partner_id,message_type,subscriber_id,created_at,signedup_at,deactivated_at,carrier_name,subscriber_zip_code,subscriber_city,timezone_default_offset,timezone_name,subscriber_state,subscriber_source,subscriber_status,subscriber_language,subscriber_deactivation_method,partner_name,partner_created_date,partner_is_active,partner_state,timezone_default_offset_adj,outbound_message_created_at_adj,outbound_message_created_at_day,outbound_message_created_at_hour,bitly_link
2674353,2c632a0d-956d-4bf4-9a2f-c656ea62071f,2020-01-14 16:04:06.141360,C1: Snacky Shapes: Let your child make shapes ...,00000000-0000-0000-0000-000000000002,normal,af9e9645-0664-449e-9e54-31e1045e8ec6,2018-01-22 12:30:24.136837,2019-12-20 16:42:30.892435,NaN,AT&T Wireless,08221,Linwood,-5.000,America/New_York,New Jersey,Text SignUp,activated,English,NaN,"WNET - New York, NY",2018-03-25 21:32:21.001262,True,New York,-1 days +19:00:00,2020-01-14 11:04:06.141360,Tuesday,11.000,bit.ly/2BE24oc
2674354,570735dd-e915-4e0b-aa12-53eb33db7219,2020-01-14 16:04:06.327875,"C8: By age three, your child should have about...",00000000-0000-0000-0000-000000000002,normal,af9e9645-0664-449e-9e54-31e1045e8ec6,2018-01-22 12:30:24.136837,2019-12-20 16:42:30.892435,NaN,AT&T Wireless,08221,Linwood,-5.000,America/New_York,New Jersey,Text SignUp,activated,English,NaN,"WNET - New York, NY",2018-03-25 21:32:21.001262,True,New York,-1 days +19:00:00,2020-01-14 11:04:06.327875,Tuesday,11.000,bit.ly/2BDYIl4
2674355,df777b26-1816-4161-a9b5-2e902d1f8399,2020-01-14 16:04:06.520980,C5: Biggest milestone for your baby? Building ...,00000000-0000-0000-0000-000000000002,normal,af9e9645-0664-449e-9e54-31e1045e8ec6,2018-01-22 12:30:24.136837,2019-12-20 16:42:30.892435,NaN,AT&T Wireless,08221,Linwood,-5.000,America/New_York,New Jersey,Text SignUp,activated,English,NaN,"WNET - New York, NY",2018-03-25 21:32:21.001262,True,New York,-1 days +19:00:00,2020-01-14 11:04:06.520980,Tuesday,11.000,bit.ly/2NXZrqZ
2674356,8f930fc4-75bd-4e2a-bd3a-d09a721ee904,2020-01-14 16:04:06.710587,C11: Having a new baby can make partners close...,00000000-0000-0000-0000-000000000002,normal,af9e9645-0664-449e-9e54-31e1045e8ec6,2018-01-22 12:30:24.136837,2019-12-20 16:42:30.892435,NaN,AT&T Wireless,08221,Linwood,-5.000,America/New_York,New Jersey,Text SignUp,activated,English,NaN,"WNET - New York, NY",2018-03-25 21:32:21.001262,True,New York,-1 days +19:00:00,2020-01-14 11:04:06.710587,Tuesday,11.000,bit.ly/2nXfpH0
2674357,b8b9a7a9-06b6-49f5-8f4b-be59fa0d8080,2020-01-14 16:04:06.904784,C5: MATH: This hands-on pattern game is lots o...,00000000-0000-0000-0000-000000000002,normal,af9e9645-0664-449e-9e54-31e1045e8ec6,2018-01-22 12:30:24.136837,2019-12-20 16:42:30.892435,NaN,AT&T Wireless,08221,Linwood,-5.000,America/New_York,New Jersey,Text SignUp,activated,English,NaN,"WNET - New York, NY",2018-03-25 21:32:21.001262,True,New York,-1 days +19:00:00,2020-01-14 11:04:06.904784,Tuesday,11.000,bit.ly/2Lu9QLm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3117832,fe3a0af8-ce50-4be7-91d9-547a667d3850,2020-01-13 16:04:16.040450,C5: Your baby needs to go to the doctor regula...,00000000-0000-0000-0000-000000000002,normal,af9e9645-0664-449e-9e54-31e1045e8ec6,2018-01-22 12:30:24.136837,2019-12-20 16:42:30.892435,NaN,AT&T Wireless,08221,Linwood,-5.000,America/New_York,New Jersey,Text SignUp,activated,English,NaN,"WNET - New York, NY",2018-03-25 21:32:21.001262,True,New York,-1 days +19:00:00,2020-01-13 11:04:16.040450,Monday,11.000,bit.ly/2BDH5C4
3117833,21d2fe4f-9b59-4ed7-afd0-ef995becec2e,2020-01-13 16:04:16.234208,C10: Imitation is very important to a child’s ...,00000000-0000-0000-0000-000000000002,normal,af9e9645-0664-449e-9e54-31e1045e8ec6,2018-01-22 12:30:24.136837,2019-12-20 16:42:30.892435,NaN,AT&T Wireless,08221,Linwood,-5.000,America/New_York,New Jersey,Text SignUp,activated,English,NaN,"

It seems that this person has or cares for a lot of kids, based on the C1-C11 stamps.

In [57]:
messages[messages['subscriber_zip_code'] == '45005']['message_type'].value_counts()

normal      246
national      3
survey        2
Name: message_type, dtype: int64

In [58]:
messages[messages['subscriber_zip_code'] == '45005']

,message_id,outbound_message_created_at,outbound_message_body,outbound_message_partner_id,message_type,subscriber_id,created_at,signedup_at,deactivated_at,carrier_name,subscriber_zip_code,subscriber_city,timezone_default_offset,timezone_name,subscriber_state,subscriber_source,subscriber_status,subscriber_language,subscriber_deactivation_method,partner_name,partner_created_date,partner_is_active,partner_state,timezone_default_offset_adj,outbound_message_created_at_adj,outbound_message_created_at_day,outbound_message_created_at_hour,bitly_link
2674631,bbb3de94-771e-40b4-bb97-9553b199b7bd,2020-01-14 16:04:30.111789,C5: El elogio es esencial para el aprendizaje....,d4d696af-37ff-427e-8035-2cb1bd630495,normal,0e93fc1c-268a-4fd5-bad4-b140d976b366,2019-04-10 14:44:57.189554,2019-04-10 14:44:57.189554,NaN,Verizon Wireless,45005,Franklin,-5.000,America/New_York,Ohio,Web SignUp,activated,Spanish,NaN,Bright by Three,2018-11-30 19:20:34.357527,True,Colorado,-1 days +19:00:00,2020-01-14 11:04:30.111789,Tuesday,11.000,bit.ly/2BCzyDj
2674632,a86cc166-e99e-4855-8623-7dea7ca13ed6,2020-01-14 16:04:30.295276,C6: El tiempo de calidad juntos puede fortalec...,d4d696af-37ff-427e-8035-2cb1bd630495,normal,0e93fc1c-268a-4fd5-bad4-b140d976b366,2019-04-10 14:44:57.189554,2019-04-10 14:44:57.189554,NaN,Verizon Wireless,45005,Franklin,-5.000,America/New_York,Ohio,Web SignUp,activated,Spanish,NaN,Bright by Three,2018-11-30 19:20:34.357527,True,Colorado,-1 days +19:00:00,2020-01-14 11:04:30.295276,Tuesday,11.000,bit.ly/2BDjuS2
2676818,06f99c6b-ff48-489c-9b20-c64e09428d29,2020-01-16 16:04:08.913639,C8: Los niños son curiosos y aprenden exploran...,d4d696af-37ff-427e-8035-2cb1bd630495,normal,0e93fc1c-268a-4fd5-bad4-b140d976b366,2019-04-10 14:44:57.189554,2019-04-10 14:44:57.189554,NaN,Verizon Wireless,45005,Franklin,-5.000,America/New_York,Ohio,Web SignUp,activated,Spanish,NaN,Bright by Three,2018-11-30 19:20:34.357527,True,Colorado,-1 days +19:00:00,2020-01-16 11:04:08.913639,Thursday,11.000,bit.ly/2BEvoLk
2676819,b52c8b57-e820-4603-80fd-a801bff89f5a,2020-01-16 16:04:09.098799,"C6: Cuando le leas a tu niño, usa voces distin...",d4d696af-37ff-427e-8035-2cb1bd630495,normal,0e93fc1c-268a-4fd5-bad4-b140d976b366,2019-04-10 14:44:57.189554,2019-04-10 14:44:57.189554,NaN,Verizon Wireless,45005,Franklin,-5.000,America/New_York,Ohio,Web SignUp,activated,Spanish,NaN,Bright by Three,2018-11-30 19:20:34.357527,True,Colorado,-1 days +19:00:00,2020-01-16 11:04:09.098799,Thursday,11.000,bit.ly/2BEWAtg
2676820,563dc86e-c99f-48e3-bb5f-422898513cbd,2020-01-16 16:04:09.277115,C7: ¡Haz el cepillarse divertido para tu bebé ...,d4d696af-37ff-427e-8035-2cb1bd630495,normal,0e93fc1c-268a-4fd5-bad4-b140d976b366,2019-04-10 14:44:57.189554,2019-04-10 14:44:57.189554,NaN,Verizon Wireless,45005,Franklin,-5.000,America/New_York,Ohio,Web SignUp,activated,Spanish,NaN,Bright by Three,2018-11-30 19:20:34.357527,True,Colorado,-1 days +19:00:00,2020-01-16 11:04:09.277115,Thursday,11.000,bit.ly/2BE3IWZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3112496,1083da99-2ae4-4e7c-9ac9-cefa23ffba50,2020-01-11 16:03:53.215875,"C7: Para tener una conversación con su hija, e...",d4d696af-37ff-427e-8035-2cb1bd630495,normal,0e93fc1c-268a-4fd5-bad4-b140d976b366,2019-04-10 14:44:57.189554,2019-04-10 14:44:57.189554,NaN,Verizon Wireless,45005,Franklin,-5.000,America/New_York,Ohio,Web SignUp,activated,Spanish,NaN,Bright by Three,2018-11-30 19:20:34.357527,True,Colorado,-1 days +19:00:00,2020-01-11 11:03:53.215875,Saturday,11.000,bit.ly/2BCzSC1
3112497,8a1aab2b-03b1-49a6-b1a7-41c138df100e,2020-01-11 16:03:53.405383,C6: Nuestro deseo de mantener a los niños limp...,d4d696af-37ff-427e-8035-2cb1bd630495,normal,0e93fc1c-268a-4fd5-bad4-b140d976b366,2019-04-10 14:44:57.189554,2019-04-10 14:44:57.189554,NaN,Verizon Wireless,45005,Franklin,-5.000,America/New_York,Ohio,Web SignUp,activated,Spanish,NaN,Bright by Three,2018-11-30 19:20:34.357527,True,Col

Same with this person.

## Linking Texts to Landing Page URL in Google Analytics

In [21]:
messages['bitly_link'] = messages['outbound_message_body'].str.extract(r'.*(bit.ly\/.*)', expand=True)
messages['bitly_link']

0          bit.ly/2BEWzWe
1          bit.ly/2uRI6Wn
2          bit.ly/2BGQA3u
3          bit.ly/2BCABTL
4          bit.ly/2BEepZm
                ...      
3554657    bit.ly/2BEKcJK
3554658    bit.ly/2BFTWU8
3554659    bit.ly/35CP3vw
3554660    bit.ly/2BEvD9c
3554661    bit.ly/32OykTQ
Name: bitly_link, Length: 3554662, dtype: object

## Tags

What's available in existing data:
- Message type
- Partner
- Program message tags (LISA, Safety, PCDC, BURNOUT, BIP LAB, BBT MATH, BBT LITERACY, ILAB, CDC, SAFETY, PBS KIDS, CDC-ORAL HEALTH, WNET, SAFECARECO, READING PARTNERS, CO IMMUNIZATION COALITION, VROOM, BIRTHDAY, SLEEP, pcdc, PBSEMOJI)
- Key in table `v_messages`
- Title in table `scheduled_messages`
- Keyword id in table `scheduled_messages` (PORTLAND, SFO, 406parents, Consejos, LISA TIPS, APTV, etc.) - this is not really used though except in old messages

Might need Lisa to clarify what these tags mean if we're going to try to do some analysis around them.

Other tags we could use:
- Key words based on message text
- "C1" and "C2," where C1 is the oldest child
- There are some tables defining various tags that are restricted to us, like `national_message_tags` and `scheduled_message_tags`. 

## Questions

- How should we analyze tags for community messages? Through `scheduled_messages_tags`? (Which we don't have access to)
- Why does the table `outbound_messages` keep changing?
- Do you have any theories about optimal content, timing, and frequency of text messages that you would like us to validate?

## Next Steps

- **Tie these high-level observations with finding optimal timing/frequency/content of messages.**
- Nim mentioned interest in doing some analysis on inbound messages to see what people are texting in, to potentially help them implement a chatbot. Maybe we can do some simple content analysis here.